In [82]:
# import bibs
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import TruncatedSVD

# Loading data

In [83]:
working_directory = os.getcwd()
path = working_directory
path

'/content'

In [84]:
gdp_data = pd.read_pickle("./data/raw/gdp.pickle")
df_pattern = pd.read_pickle("./data/raw/naics_pattern.pickle")

df_occupation_1 = pd.read_pickle("./data/raw/naics_occupation_part1.pickle")
df_occupation_2 = pd.read_pickle("./data/raw/naics_occupation_part2.pickle")
df_occupation_3 = pd.read_pickle("./data/raw/naics_occupation_part3.pickle")

In [85]:
df_state = pd.read_pickle("./data/raw/state.pickle")
df_county = pd.read_pickle("./data/raw/county.pickle")

# Data Exploration & Preprocessing

In [86]:
df_pattern['FIPS'] = df_pattern['FIPS'].astype(str)
unique_lengths = df_pattern['FIPS'].apply(len).unique()
unique_lengths

array([4, 5])

In [87]:
def add_zeros(code):
    code = str(code)
    if len(code) == 3:
        return '00' + code
    elif len(code) == 4:
        return '0' + code
    elif len(code) == 1:
        return '0000' + code
    return code

In [88]:
df_pattern['FIPS'] = df_pattern['FIPS'].apply(add_zeros)

In [89]:
df_occupation = pd.concat([df_occupation_1, df_occupation_2, df_occupation_3], ignore_index=True)
df_occupation.head(2)

,FIPS,State_GEOID,naics,NAICS_TITLE,emp_total_county_naics,OCC_CODE,OCC_TITLE,emp_occupation,state_name
0,12999,12,5613,Employment Services,1436559,49-9071,"Maintenance and Repair Workers, General",20639.514235,NaN
1,06999,6,5613,Employment Services,729335,49-9071,"Maintenance and Repair Workers, General",9414.167765,NaN


In [90]:
df_state.head(2).sort_values(by='STATEFP', ascending=False)

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21524, -88.49176 ..."


In [91]:
df_county.head(2).sort_values(by='STATEFP', ascending=False)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.0463, -89.17938 37.0530..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."


In [92]:
filtered_statefp = df_state.loc[~df_state['NAME'].isin(['Alaska', 'Hawaii', 'Puerto Rico', 'Commonwealth of the Northern Mariana Islands', 'American Samoa', 'United States Virgin Islands', 'Guam']), 'STATEFP']

filtered_state_df = df_state[df_state['STATEFP'].isin(filtered_statefp)]
filtered_county_df = df_county[df_county['STATEFP'].isin(filtered_statefp)]

In [93]:
filtered_county_df = filtered_county_df.rename(columns={"GEOID": "FIPS"})
filtered_county_df = filtered_county_df.rename(columns={"geometry": "county_geometry"})
filtered_state_df = filtered_state_df.rename(columns={"geometry": "state_geometry"})
filtered_county_df = filtered_county_df.rename(columns={"NAME": "county_name"})
filtered_state_df = filtered_state_df.rename(columns={"NAME": "state_name"})

In [94]:
filtered_county_df = filtered_county_df[["STATEFP", "county_geometry", "FIPS", "county_name"]]
filtered_state_df = filtered_state_df[["STATEFP", "state_geometry", "state_name"]]

**1. Selection of features:**

You must select the right features (columns) that have the greatest impact on the main outcome of the project (**sales of tools for the metal industry**). There should be a causal relationship between the features and sales.

<u>Your task will be to select features from the following areas:<u>
- Employment
- Establishment
- Payroll
- Economic strength


## Data cleaning

In [95]:
gdp_data.isnull().sum()

,0
FIPS,0
GeoName,0
Region,0
TableName,0
LineCode,0
IndustryClassification,0
Description,0
Unit,0
2017,18106
2018,18041


In [96]:
# delete null + zero (outliers) data
columns_to_edit = ["2017", "2018", "2019", "2020", "2021", "2022"]
for c in columns_to_edit:
    gdp_data = gdp_data[gdp_data[c].notna()]
    gdp_data = gdp_data[gdp_data[c] != 0]

# False is the count of valid data
gdp_data.apply(lambda x: x.isnull().value_counts())

,FIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2017,2018,2019,2020,2021,2022
False,75208,75208,75208,75208,75208,75208,75208,75208,75208,75208,75208,75208,75208,75208


In [97]:
# There's no missing data in df_pattern
df_pattern.isnull().sum()

,0
State_GEOID,0
County_GEOID,0
FIPS,0
naics_2,0
naics,0
DESCRIPTION,0
emp_nf,0
emp,0
qp1_nf,0
qp1,0


In [98]:
# we had null state names, I decided not to delete the entire rows but to fill the names with "Unknown" values
df_occupation["state_name"] = df_occupation["state_name"].fillna("Unknown")
df_occupation.isnull().sum()

,0
FIPS,0
State_GEOID,0
naics,0
NAICS_TITLE,0
emp_total_county_naics,0
OCC_CODE,0
OCC_TITLE,0
emp_occupation,0
state_name,0


In [99]:
gdp_data['Description'] = gdp_data['Description'].str.strip()

## Features selection

In [100]:
# get naics related to metal industries
metal_naics_codes = ['332', '333']

In [101]:
# get employment occupation data in metal industries

# I kept the most important columns. If you need any other columns add them to groupby and when merge
# but this may result in redundunt number of rows. E.g. if we add OCC_TITLE, we may end up with many simillar rows that differ only in OCC_TITLE
occupation_summary = df_occupation[df_occupation['naics'].str.startswith(tuple(metal_naics_codes))].groupby(['FIPS', 'naics']).agg(
    emp_occupation=('emp_occupation', 'sum'),
    OCC_TITLE=('OCC_TITLE', list)
).sort_values(by='emp_occupation', ascending=False)
occupation_summary

emp_occupation  \
FIPS  naics                    
48201 3330A1     9173.489484   
      3320A2     6204.545922   
06037 3327       5950.821371   
26099 3335       5359.304915   
48201 3320A1     4835.324750   
...                      ...   
06009 3320A2        0.740715   
49043 3327          0.485070   
48165 3330A1        0.373060   
04005 3320A1        0.327316   
15003 3328          0.315683   

                                                      OCC_TITLE  
FIPS  naics                                                      
48201 3330A1  [Welders, Cutters, Solderers, and Brazers, Mac...  
      3320A2  [Welders, Cutters, Solderers, and Brazers, Str...  
06037 3327    [Machinists, Grinding, Lapping, Polishing, and...  
26099 3335    [Machinists, Tool and Die Makers, Mechanical E...  
48201 3320A1  [Welders, Cutters, Solderers, and Brazers, Cut...  
...                                                         ...  
06009 3320A2  [Welders, Cutters, Solderers, and Brazers, She...  
49043 3327    [Machinists, Welders, Cutters, Solderers, and ...  
48165 3330A1  [Welders, Cutters, Solderers, and Brazers, Mec...  
04005 3320A1  [Welders, Cutters, Solderers, and Brazers, Mac...  
15003 3328    [Maintenance and Repair Workers, General, Indu...  

[4748 rows x 2 columns]

In [102]:
# The same here with establishments
# Features 'ap' and 'qp1' are about payroll. They are highly correlated, so it's maybe enough to keep only annual payroll
metal_industry = df_pattern[df_pattern['naics'].str.startswith(tuple(metal_naics_codes))]
regional_est_summary = metal_industry.groupby(['FIPS', 'naics', 'emp', 'ap', 'DESCRIPTION']).agg(
    est=('est', 'sum'),
).sort_values(by='est', ascending=False)
regional_est_summary

,,,,,est
FIPS,naics,emp,ap,DESCRIPTION,
06037,3327,12485,747445,"Machine Shops; Turned Product; and Screw, Nut, and Bolt Manufacturing",762
06059,3327,8903,551109,"Machine Shops; Turned Product; and Screw, Nut, and Bolt Manufacturing",483
48201,3327,6561,405512,"Machine Shops; Turned Product; and Screw, Nut, and Bolt Manufacturing",460
06037,3320A2,7134,468887,"3323, 3323",370
17031,3327,6853,458457,"Machine Shops; Turned Product; and Screw, Nut, and Bolt Manufacturing",342
...,...,...,...,...,...
20099,3320A1,75,3969,"3321, 3322, 3325, 3326, 3329",3
20103,3330A1,266,12452,"3331, 3332, 3334, 3339",3
20113,3320A1,270,13082,"3321, 3322, 3325, 3326, 3329",3


In [115]:
gdp_data['FIPS'] = gdp_data['FIPS'].apply(add_zeros)

gdp_by_fips = gdp_data.groupby(['FIPS']).agg(
    gdp_2017=('2017', 'sum'),
    gdp_2018=('2018', 'sum'),
    gdp_2019=('2019', 'sum'),
    gdp_2020=('2020', 'sum'),
    gdp_2021=('2021', 'sum'),
    gdp_2022=('2022', 'sum')
).reset_index()

# count new feature - gdp growth rate
gdp_by_fips['gdp_growth_rate'] = (
    (gdp_by_fips['gdp_2022'] - gdp_by_fips['gdp_2017']) / gdp_by_fips['gdp_2017']
) * 100
gdp_by_fips.sort_values(by='gdp_growth_rate', ascending=False)

,FIPS,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_growth_rate
2643,48155,92267.0,150870.0,170041.0,297734.0,318308.0,318538.0,245.235024
1649,30051,331001.0,819405.0,971255.0,974640.0,910523.0,1045909.0,215.983638
2613,48095,282507.0,303471.0,404183.0,459189.0,483080.0,700012.0,147.785719
2443,46083,11710466.0,12389027.0,13343873.0,21155213.0,23699998.0,28948859.0,147.205013
2606,48081,300597.0,318402.0,308262.0,314668.0,641347.0,693350.0,130.657658
...,...,...,...,...,...,...,...,...
2807,48483,3245700.0,2749311.0,2410058.0,2169011.0,1891568.0,1454212.0,-55.195736
254,08009,656182.0,491757.0,356075.0,366075.0,455467.0,290235.0,-55.769131
1742,31123,3063544.0,2951285.0,2361315.0,2429658.0,1360632.0,1318718.0,-56.954495
2168,40003,3145676.0,2177536.0,1712713.0,1809021.0,1505726.0,1298068.0,-58.734847


In [116]:
# merge tables
merged_occupation_est = occupation_summary.reset_index().merge(
    regional_est_summary.reset_index(),
    on=['FIPS', 'naics'],
    how='inner'
)

occupation_agg = merged_occupation_est.groupby(['FIPS', 'naics', 'DESCRIPTION']).agg(
    emp_occupation=('emp_occupation', 'sum'),
    est=('est', 'sum'),
    emp=('emp', 'sum'),
    ap=('ap', 'sum'),
).reset_index()

final_table = occupation_agg.merge(
    gdp_by_fips[['FIPS', 'gdp_growth_rate', 'gdp_2022']],
    on='FIPS',
    how='inner'
)

filtered_county_df['STATEFP'] = filtered_county_df['STATEFP'].astype(str)
filtered_state_df['STATEFP'] = filtered_state_df['STATEFP'].astype(str)

final_table = final_table.merge(
    filtered_county_df,
    on='FIPS',
    how='inner'
)

final_table = final_table.merge(
    filtered_state_df,
    on='STATEFP',
    how='inner'
)


final_table.sort_values(
    by='gdp_growth_rate', ascending=False
)

,FIPS,naics,DESCRIPTION,emp_occupation,est,emp,ap,gdp_growth_rate,gdp_2022,STATEFP,county_geometry,county_name,state_geometry,state_name
3779,46083,3330A1,"3331, 3332, 3334, 3339",96.062610,3,105,5071,147.205013,28948859.0,46,"POLYGON ((-96.92489 43.47566, -96.92484 43.500...",Lincoln,"POLYGON ((-104.05788 44.9976, -104.05078 44.99...",South Dakota
3778,46083,3327,"Machine Shops; Turned Product; and Screw, Nut,...",18.561262,5,18,964,147.205013,28948859.0,46,"POLYGON ((-96.92489 43.47566, -96.92484 43.500...",Lincoln,"POLYGON ((-104.05788 44.9976, -104.05078 44.99...",South Dakota
3777,46083,3320A2,"3323, 3323",420.689383,9,405,25428,147.205013,28948859.0,46,"POLYGON ((-96.92489 43.47566, -96.92484 43.500...",Lincoln,"POLYGON ((-104.05788 44.9976, -104.05078 44.99...",South Dakota
4051,48187,3327,"Machine Shops; Turned Product; and Screw, Nut,...",38.929188,10,77,3720,72.549052,51521035.0,48,"POLYGON ((-98.31093 29.59447, -98.2969 29.5998...",Guadalupe,"MULTIPOLYGON (((-94.7183 29.72886, -94.71721 2...",Texas
4050,48187,3320A2,"3323, 3323",164.565806,12,299,14190,72.549052,51521035.0,48,"POLYGON ((-98.31093 29.59447, -98.2969 29.5998...",Guadalupe,"MULTIPOLYGON (((-94.7183 29.72886, -94.71721 2...",Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,48097,3327,"Machine Shops; Turned Product; and Screw, Nut,...",95.036527,8,188,10473,-29.895020,11549937.0,48,"POLYGON ((-97.48697 33.44819, -97.4869 33.4587...",Cooke,"MULTIPOLYGON (((-94.7183 29.72886, -94.71721 2...",Texas
3986,48097,3320A2,"3323, 3323",11.006707,7,20,936,-29.895020,11549937.0,48,"POLYGON ((-97.48697 33.44819, -97.4869 33.4587...",Cooke,"MULTIPOLYGON (((-94.7183 29.72886, -94.71721 2...",Texas
1606,22093,3327,"Machine Shops; Turned Product; and Screw, Nut,...",98.969380,4,135,12879,-34.716140,9205213.0,22,"POLYGON ((-90.96369 30.06645, -90.9356 30.0857...",St. James,"MULTIPOLYGON (((-88.8677 29.86155, -88.86566 2...",Louisiana
3338,40147,3320A2,"3323, 3323",41.926345,5,71,4516,-41.626858,6638743.0,40,"POLYGON ((-96.00114 36.43976, -96.00108 36.448...",Washington,"POLYGON ((-103.00256 36.52659, -103.00219 36.6...",Oklahoma


## Filtering & Aggregation

**2. Filtering:**

Identify the most important occupations and industries that have a **realistically high tool consumption**

Important KPIs for determining high tool consumption:


1.   **emp_occupation**: Determines the amount of employees in a county working in the metall processing industry.
2.   **gdp_growth_rate**: determines how the GDP develops in a specific county.
3.  **gdp_2022**: is the most recent GDP from the year 2022.






## Scaling

In [130]:
mm_scaler = MinMaxScaler()
df_final = final_table.copy()

columns_to_scale = ['gdp_2022', 'gdp_growth_rate', 'emp_occupation']
scaled_columns = [col + '_scaled' for col in columns_to_scale]
df_final[scaled_columns] = mm_scaler.fit_transform(df_final[columns_to_scale])

df_final.head()

,FIPS,naics,DESCRIPTION,emp_occupation,est,emp,ap,gdp_growth_rate,gdp_2022,STATEFP,county_geometry,county_name,state_geometry,state_name,gdp_2022_scaled,gdp_growth_rate_scaled,emp_occupation_scaled
0,01003,3320A1,"3321, 3322, 3325, 3326, 3329",44.574567,5,72,4638,21.477243,41628984.0,01,"POLYGON ((-88.02858 30.22676, -88.02399 30.230...",Baldwin,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",Alabama,0.010978,0.363931,0.001206
1,01003,3320A2,"3323, 3323",88.354541,16,122,7741,21.477243,41628984.0,01,"POLYGON ((-88.02858 30.22676, -88.02399 30.230...",Baldwin,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",Alabama,0.010978,0.363931,0.002399
2,01003,3327,"Machine Shops; Turned Product; and Screw, Nut,...",56.341285,10,85,3981,21.477243,41628984.0,01,"POLYGON ((-88.02858 30.22676, -88.02399 30.230...",Baldwin,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",Alabama,0.010978,0.363931,0.001527
3,01003,3330A1,"3331, 3332, 3334, 3339",132.428720,4,247,18297,21.477243,41628984.0,01,"POLYGON ((-88.02858 30.22676, -88.02399 30.230...",Baldwin,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",Alabama,0.010978,0.363931,0.003600
4,01007,3327,"Machine Shops; Turned Product; and Screw, Nut,...",10.607101,3,16,835,12.137373,1647880.0,01,"POLYGON ((-87.42194 33.00338, -87.31854 33.006...",Bibb,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",Alabama,0.000379,0.316679,0.000280


## Ranking

In [134]:
# Defining the weights based on the importance of the KPI for determining high tool consumption
emp_occ_weight = 0.6 #many metal working employees lead to high tool consumption
gdp_growth_weight = 0.1  #high GDP growth can provide implications of the future for the metal industry in a specific county
gdp_weight = 0.3 #recent high GDP indicates high profit for the metal industry in a specific county

In [135]:
df_final['weighted_score'] = (
    df_final['emp_occupation_scaled'] * emp_occ_weight +
    df_final['gdp_growth_rate_scaled'] * gdp_growth_weight +
    df_final['gdp_2022_scaled'] * gdp_weight
)

df_final['ranking'] = df_final['weighted_score'].rank(ascending=False, method='min').astype(int)

df_final = df_final.sort_values(by='ranking')
df_final.head()

,FIPS,naics,DESCRIPTION,emp_occupation,est,emp,ap,gdp_growth_rate,gdp_2022,STATEFP,county_geometry,county_name,state_geometry,state_name,gdp_2022_scaled,gdp_growth_rate_scaled,emp_occupation_scaled,weighted_score,ranking
4057,48201,3330A1,"3331, 3332, 3334, 3339",36693.957934,306,22733,1658815,6.851240,1.955070e+09,48,"MULTIPOLYGON (((-94.97839 29.68365, -94.97744 ...",Harris,"MULTIPOLYGON (((-94.7183 29.72886, -94.71721 2...",Texas,0.518212,0.289936,1.000000,0.784457,1
252,06037,3320A1,"3321, 3322, 3325, 3326, 3329",20595.649003,359,11567,739506,10.498052,3.772520e+09,06,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",Los Angeles,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",California,1.000000,0.308386,0.561278,0.667605,2
4053,48201,3320A1,"3321, 3322, 3325, 3326, 3329",24176.623749,293,10150,738324,6.851240,1.955070e+09,48,"MULTIPOLYGON (((-94.97839 29.68365, -94.97744 ...",Harris,"MULTIPOLYGON (((-94.7183 29.72886, -94.71721 2...",Texas,0.518212,0.289936,0.658869,0.579779,3
937,17031,3320A1,"3321, 3322, 3325, 3326, 3329",23691.144795,252,8885,592428,5.706606,1.998678e+09,17,"POLYGON ((-88.26364 42.06687, -88.25835 42.066...",Cook,"POLYGON ((-91.51297 40.18106, -91.51107 40.188...",Illinois,0.529772,0.284146,0.645638,0.574729,4
256,06037,3330A1,"3331, 3332, 3334, 3339",9353.724951,303,7736,553527,10.498052,3.772520e+09,06,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",Los Angeles,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",California,1.000000,0.308386,0.254905,0.483782,5


## Aggregating

In [143]:
# Creating table for top counties
aggregated_counties = df_final.groupby(['county_name', 'state_name'], as_index=False).agg(
    total_weighted_score=('weighted_score', 'sum'),
    average_rank=('ranking', 'mean'),
    avg_gdp_2022=('gdp_2022', 'mean'),
    avg_gdp_growth_rate=('gdp_growth_rate', 'mean'),
    avg_emp_occupation=('emp_occupation', 'mean')
)

aggregated_counties = aggregated_counties.sort_values(by='total_weighted_score', ascending=False)

top_10_counties = aggregated_counties.head(10)
top_10_counties

,county_name,state_name,total_weighted_score,average_rank,avg_gdp_2022,avg_gdp_growth_rate,avg_emp_occupation
832,Los Angeles,California,3.365051,7.625,3.772520e+09,10.498052,5491.695148
586,Harris,Texas,2.671710,37.125,1.955070e+09,6.851240,9143.554062
320,Cook,Illinois,2.173481,37.625,1.998678e+09,5.706606,5158.162386
1208,Santa Clara,California,1.738421,44.750,1.954491e+09,38.385599,1036.179669
727,King,Washington,1.636389,56.125,1.798052e+09,34.873815,1125.698340
869,Maricopa,Arizona,1.614862,63.875,1.422889e+09,26.986968,3029.779023
352,Dallas,Texas,1.452368,70.000,1.449368e+09,21.841304,1818.018558
1039,Orange,California,1.431083,75.000,1.316700e+09,11.843951,2609.851068
1001,New York,New York,1.370177,21.000,3.052353e+09,11.186269,7.750215
1196,San Diego,California,1.175239,97.750,1.170345e+09,17.367376,1194.968104


# Clustering

**3. Feature engineering / construction / aggregation:**

In all three datasets there are duplicates in the FIPS column as further dimensions are mapped in the naics or occ_code columns. This makes specific combinations of FIPS, NAICS, Occupation and a specific feature in a row possible. <br>

Due to the requirement to use a **cluster algorithm**, the final dataframe must have a special form. **Each region (FIPS number) may only occur once in the df.**
You must therefore prioritize the industries and occupations in this step and create special features that contain the Occupation or Industry dimension within them.

This is called **Pivoting or Wide-Format-Transformation**

<u>Possible features could be as listed below:<u>

- Number of employees in foundries (NAICS: 3315)
- Number of grinders (OCC: 51-9022)
- Number of grinders (OCC: 51-9022) in foundries (NAICS: 3315)